### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
OZ project Project<br>
-- Q_3p Cross Sections Examples
</H1>

<b>This notebook will fit impact ionisation cross sections for</b> 
    
 > - <b>H(n=3)* + p = p* + p + e</b> <br>
> Taken from Janev et al 1993 IAEA atomic data tables. They provide fits, so here I just use their results and plot to make sure they extend to higher energies in a sensible way. I tried my own fits here, but their fit works
well already. 

 > - <b>H(n=3)* + H = p* + H + e</b> <br>
 > ?????
 
 > - <b>H(n=3)* + E = p* + H + e</b> <br>
> Taken from Janev et al 1993 IAEA atomic data tables. They provide fits, so here I just use their results and plot to make sure they extend to higher energies in a sensible way. I tried my own fits here, but their fit works
well already.  
 





---
### <b style="color:blue"> Some set up </b>

***Import Modules***

In [ ]:
##
## Import some modules
##

import sys
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/radynpy/')
sys.path.insert(0,'/Users/gskerr1/Documents/Research/OrrallZirkerEffect/')


import radynpy
import OrrallZirkerPy as OZpy
# from OrrallZirkerPy import CrossSections
from OrrallZirkerPy.CrossSections import chebyshev_fn as Cheb
from numpy.polynomial.polynomial import Polynomial as Poly



import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from matplotlib.ticker import LogLocator
from matplotlib import ticker
import matplotlib.colorbar as cb
import pandas as pd

import cmocean
import colorcet as ccet
import palettable as pal 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
import importlib
importlib.reload(OZpy.CrossSections)

***Set up plot properties***

In [ ]:
##
## Plot properties (these are just personal preference)
##

#Avenir LT Std
font = {'family': 'Avenir LT Std',
        'color':  'black',
        'weight': 'medium',
        'size': 22,
        }
plot_params = {'ytick.direction': 'in', 
               'xtick.direction': 'in', 
               'xtick.minor.visible': True,
               'ytick.minor.visible': True,
               'xtick.major.size': 10, 'xtick.minor.size': 5,
               'ytick.major.size': 10, 'ytick.minor.size': 5,
               'ytick.right': False,
               'xtick.top': False,
               'ytick.major.width': 1.5,
               'xtick.major.width': 1.5,
               'ytick.minor.width': 1.5,
               'xtick.minor.width': 1.5,
               'axes.linewidth': 1.5,
               'axes.spines.top': False,
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': False,
               'axes.titlepad' : 18 }

plot_lg_params = {'legend.frameon': False}
#plt.rcParams.update({'font.size': font['size'], 'font.family':font['family'], 'font.weight':font['weight'], 'font.color':font['color']})

plt.rcParams.update({'font.size':font['size'], 'font.family':font['family'], 'font.weight':font['weight']})
plt.rcParams.update({'ytick.direction': plot_params['ytick.direction'],
                     'xtick.direction': plot_params['xtick.direction'],
                     'xtick.minor.visible': plot_params['xtick.minor.visible'],
                     'ytick.minor.visible': plot_params['ytick.minor.visible'],
                     'ytick.major.size':  plot_params['ytick.major.size'], 
                     'ytick.minor.size':  plot_params['ytick.minor.size'],
                     'xtick.major.size':  plot_params['xtick.major.size'],                                
                     'xtick.minor.size':  plot_params['xtick.minor.size'],
                     'ytick.right': plot_params['ytick.right'],
                     'xtick.top': plot_params['xtick.top'],
                     'ytick.major.width': plot_params['ytick.major.width'],
                     'xtick.major.width': plot_params['xtick.major.width'],
                     'ytick.minor.width': plot_params['ytick.minor.width'],
                     'xtick.minor.width': plot_params['xtick.minor.width'],                    
                     'axes.linewidth': plot_params['axes.linewidth'],
                     'axes.spines.top' : plot_params['axes.spines.top'],
                     'axes.spines.bottom' : plot_params['axes.spines.bottom'],
                     'axes.spines.left' : plot_params['axes.spines.left'],
                     'axes.spines.right' : plot_params['axes.spines.right'],
                     'axes.titlepad' : plot_params['axes.titlepad'],
                    })

plt.rcParams.update({'legend.frameon': plot_lg_params['legend.frameon']})

mpl.mathtext.SHRINK_FACTOR = 0.6
mpl.mathtext.GROW_FACTOR = 1 / 0.6





template = dict(
        layout = go.Layout(font = dict(family = "Rockwell", size = 18),
                           title_font = dict(family = "Rockwell", size = 22), 
                           plot_bgcolor = 'white',
                           paper_bgcolor = 'white',
                           xaxis = dict(
                                showexponent = 'all',
                                exponentformat = 'e',
                                tickangle = 0,
                                linewidth = 4,
                                showgrid = True,
                            ),
                            yaxis = dict(
                          showexponent = 'all',
                          exponentformat = 'e',
                                linewidth = 4,
                                showgrid = True,
                                anchor = 'free',
                                position = 0,
                                domain = [0.0,1]
                            ),
                            coloraxis_colorbar = dict(
                                thickness = 15,
                                tickformat = '0.2f',
                                ticks = 'outside',
                                titleside = 'right'
                            )
                            ))

---
### <b style="color:blue"> Calculate or extract the cross sections </b>

***Set up the CrossSec object with requested energy values***

In [ ]:
energy = np.arange(1,8001,1)
energy_low = np.arange(20/1e3, 1e6/100, 1e-3)
cs = OZpy.CrossSections.CrossSec(energy)
cs_low = OZpy.CrossSections.CrossSec(energy_low)

In [ ]:
##
## These are the data that will be fit
##
iaea93_Q_3pP = OZpy.CrossSections.cs_iaea93_Q_3pP()
iaea93_Q_3pE = OZpy.CrossSections.cs_iaea93_Q_3pE()

kerr = cs.cs_kerr_poly()
kerr_ch = cs.cs_kerr_cheb()
# kerr_low= cs_low.cs_kerr_poly()
# kerr_ch_low = cs_low.cs_kerr_cheb()

***Plot underlying data for sanity checks***



> - Q_3pP

In [ ]:
plt.plot(kerr.energy, np.log10(kerr.Q_3pP), 
         linestyle = '-', linewidth = 3, color = 'tomato')
plt.plot(iaea93_Q_3pP.energy, np.log10(iaea93_Q_3pP.Q_3pP), 
         linestyle = 'none', marker = 'd', markersize = 8, color = 'darkblue')

plt.xscale('log')
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')

# plt.axvline(x=500)
# plt.axvline(x=1500)

> - Q_3pE

In [ ]:
plt.plot(kerr_low.energy, (np.log10(kerr_low.Q_3pE)), 
         linestyle = '-', linewidth = 3, color = 'pink')
plt.plot(kerr.energy, (np.log10(kerr.Q_3pE)), 
         linestyle = '--', linewidth = 3, color = 'tomato')
plt.plot(iaea93_Q_3pE.energy, np.log10(iaea93_Q_3pE.Q_3pE), 
         linestyle = 'none', marker = 'd', markersize = 8, color = 'darkblue')


plt.xscale('log')
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')

# plt.axvline(x=500)
# plt.axvline(x=1500)

---
### <b style="color:blue"> Do the fits </b>

***Fit the various cross sections***

> - Q_3pP

In [ ]:
e1_P = 5e2/1e3
e2_P = 5e6/1e3

cfit_Q3pP_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pP.energy, 
                                                          iaea93_Q_3pP.Q_3pP, emin = e1_P, emax = e2_P, deg = 4)
cfit_Q3pP_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pP.energy, 
                                                          iaea93_Q_3pP.Q_3pP, emin = e1_P, emax = e2_P, deg = 5)
cfit_Q3pP_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pP.energy, 
                                                          iaea93_Q_3pP.Q_3pP, emin = e1_P, emax = e2_P, deg = 6)
cfit_Q3pP_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pP.energy, 
                                                          iaea93_Q_3pP.Q_3pP, emin = e1_P, emax = e2_P, deg = 7)
cfit_Q3pP_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pP.energy, 
                                                          iaea93_Q_3pP.Q_3pP, emin = e1_P, emax = e2_P, deg = 8)

pfit_Q3pP_4deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pP.energy, iaea93_Q_3pP.Q_3pP,  
                                                       order = 4, log10E = True, log10Q = True)
pfit_Q3pP_5deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pP.energy, iaea93_Q_3pP.Q_3pP,  
                                                       order = 5, log10E = True, log10Q = True)
pfit_Q3pP_6deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pP.energy, iaea93_Q_3pP.Q_3pP,  
                                                       order = 6, log10E = True, log10Q = True)
pfit_Q3pP_7deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pP.energy, iaea93_Q_3pP.Q_3pP,  
                                                       order = 7, log10E = True, log10Q = True)
pfit_Q3pP_8deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pP.energy, iaea93_Q_3pP.Q_3pP,  
                                                       order = 8, log10E = True, log10Q = True)

In [ ]:
##
## Do the straight line fit to the high energy range
##
estart = np.abs(iaea93_Q_3pP.energy - 5e5/1e3).argmin()
eend = np.abs(iaea93_Q_3pP.energy - 5e6).argmin()
energy2fit = iaea93_Q_3pP.energy[estart:eend+1]
csec2fit = iaea93_Q_3pP.Q_3pP[estart:eend+1]

pfit_Q3pP_2deg_end = OZpy.CrossSections.cs_polyfit(energy2fit, csec2fit,  
                                                       order = 2, log10E = True, log10Q = True)
pfit_Q3pP_1deg_end = OZpy.CrossSections.cs_polyfit(energy2fit, csec2fit,  
                                                       order = 1, log10E = True, log10Q = True)




In [ ]:
##
## Plot the Q_3pP results (poly)
##
energy3plot = np.arange(5e2, 5e6, 1)/1e3
plt.plot((energy3plot), 
                   (pfit_Q3pP_4deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'black',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_5deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_6deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'dodgerblue',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_7deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'forestgreen',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_8deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'goldenrod'
                 )
plt.plot(iaea93_Q_3pP.energy, np.log10(iaea93_Q_3pP.Q_3pP), marker = 'd', markersize = 8, color = 'dimgrey')

energy3plot = np.arange(4e3, 10001,1)
plt.plot((energy3plot), 
                   (pfit_Q3pP_2deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_1deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'dodgerblue',
                 )

plt.plot(energy, np.log10(kerr.Q_3pP), color = 'black', linestyle = '--', linewidth = 3)

# plt.xlim([5,20])
# plt.ylim([-1,0])
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')

In [ ]:
##
## Plot the Q_3pP results (cheb)
##

energy3plot = np.arange(5e2, 5e6, 1)/1e3
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_4deg, len(cfit_Q3pP_4deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '-', color = 'black',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_5deg, len(cfit_Q3pP_5deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '-', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_6deg, len(cfit_Q3pP_6deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '-', color = 'dodgerblue',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_7deg, len(cfit_Q3pP_7deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '-', color = 'forestgreen',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_8deg, len(cfit_Q3pP_8deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '-', color = 'goldenrod'
                 )
plt.plot(iaea93_Q_3pP.energy, np.log10(iaea93_Q_3pP.Q_3pP), marker = 'd', markersize = 8, color = 'dimgrey')

energy3plot = np.arange(4e3, 10001,1)
plt.plot((energy3plot), 
                   (pfit_Q3pP_2deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pP_1deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'dodgerblue',
                 )

plt.plot(energy, np.log10(kerr.Q_3pP), color = 'black', linestyle = '--', linewidth = 3)

# plt.xlim([5,20])
# plt.ylim([-1,0])
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')


# plt.plot(energy, np.log10(kerr.Q_1pP),linewidth=2, color = 'forestgreen')
# plt.plot(energy, np.log10(kerr_ch.Q_1pP),linewidth=2, color = 'red')

In [ ]:
## 
## Compare the polynomial to chebyshev fit 
##

energy3plot = np.arange(1, 10001, 1)


plt.plot((energy3plot), 
                   (pfit_Q3pP_8deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'black'
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pP_8deg, len(cfit_Q3pP_8deg)-1, e1_P, e2_P)))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot(kerr.energy, np.log10(kerr.Q_3pP),linestyle = '-',linewidth=4, color = 'goldenrod')
plt.plot(iaea93_Q_3pP.energy, np.log10(iaea93_Q_3pP.Q_3pP), marker = 'd', markersize = 5, color = 'dodgerblue')


plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')
# plt.ylim([-1.5, 1.2])

> - Q_3pE

In [ ]:
e1_E = 2e0/1e3
e2_E = 1e4/1e3

cfit_Q3pE_4deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pE.energy, 
                                                          iaea93_Q_3pE.Q_3pE, emin = e1_E, emax = e2_E, deg = 4)
cfit_Q3pE_5deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pE.energy,
                                                          iaea93_Q_3pE.Q_3pE, emin = e1_E, emax = e2_E, deg = 5)
cfit_Q3pE_6deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pE.energy, 
                                                          iaea93_Q_3pE.Q_3pE, emin = e1_E, emax = e2_E, deg = 6)
cfit_Q3pE_7deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pE.energy, 
                                                          iaea93_Q_3pE.Q_3pE, emin = e1_E, emax = e2_E, deg = 7)
cfit_Q3pE_8deg, pcov_cheb = OZpy.CrossSections.cs_chebfit(iaea93_Q_3pE.energy, 
                                                          iaea93_Q_3pE.Q_3pE, emin = e1_E, emax = e2_E, deg = 8)

pfit_Q3pE_4deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pE.energy, iaea93_Q_3pE.Q_3pE,  
                                                       order = 4, log10E = True, log10Q = True)
pfit_Q3pE_5deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pE.energy, iaea93_Q_3pE.Q_3pE,  
                                                       order = 5, log10E = True, log10Q = True)
pfit_Q3pE_6deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pE.energy, iaea93_Q_3pE.Q_3pE,  
                                                       order = 6, log10E = True, log10Q = True)
pfit_Q3pE_7deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pE.energy, iaea93_Q_3pE.Q_3pE,  
                                                       order = 7, log10E = True, log10Q = True)
pfit_Q3pE_8deg = OZpy.CrossSections.cs_polyfit(iaea93_Q_3pE.energy, iaea93_Q_3pE.Q_3pE,  
                                                       order = 8, log10E = True, log10Q = True)

In [ ]:
##
## Do the straight line fit to the high energy range
##
estart = np.abs(iaea93_Q_3pE.energy - 1e3/1e3).argmin()
eend = np.abs(iaea93_Q_3pE.energy - 1e4/1e3).argmin()
energy2fit = iaea93_Q_3pE.energy[estart:eend+1]
csec2fit = iaea93_Q_3pE.Q_3pE[estart:eend+1]

pfit_Q3pE_2deg_end = OZpy.CrossSections.cs_polyfit(energy2fit, csec2fit,  
                                                       order = 2, log10E = True, log10Q = True)
pfit_Q3pE_1deg_end = OZpy.CrossSections.cs_polyfit(energy2fit, csec2fit,  
                                                       order = 1, log10E = True, log10Q = True)




In [ ]:
##
## Plot the Q_3pE results (poly)
##
energy3plot = np.arange(2e0, 1e4, 1)/1e3
plt.plot((energy3plot), 
                   (pfit_Q3pE_4deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'black',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_5deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_6deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'dodgerblue',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_7deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'forestgreen',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_8deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'goldenrod'
                 )
plt.plot(iaea93_Q_3pE.energy, np.log10(iaea93_Q_3pE.Q_3pE), marker = 'd', markersize = 8, color = 'dimgrey')

energy3plot = np.arange(10, 10001,1)
plt.plot((energy3plot), 
                   (pfit_Q3pE_2deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_1deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'dodgerblue',
                 )

plt.plot(energy, np.log10(kerr.Q_3pE), color = 'black', linestyle = '--', linewidth = 3)

# plt.xlim([5,20])
# plt.ylim([-3,2])
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')

In [ ]:
#
## Plot the Q_3pE results (cheb)
##

energy3plot = np.arange(4e0, 1e4, 1)/1e3
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_4deg, len(cfit_Q3pE_4deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '-', color = 'black',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_5deg, len(cfit_Q3pE_5deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '-', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_6deg, len(cfit_Q3pE_6deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '-', color = 'dodgerblue',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_7deg, len(cfit_Q3pE_7deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '-', color = 'forestgreen',
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_8deg, len(cfit_Q3pE_8deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '-', color = 'goldenrod'
                 )
plt.plot(iaea93_Q_3pE.energy, np.log10(iaea93_Q_3pE.Q_3pE), marker = 'd', markersize = 8, color = 'dimgrey')

energy3plot = np.arange(10, 10001,1)
plt.plot((energy3plot), 
                   (pfit_Q3pE_2deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot((energy3plot), 
                   (pfit_Q3pE_1deg_end(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '--', color = 'dodgerblue',
                 )

plt.plot(energy, np.log10(kerr.Q_3pE), color = 'black', linestyle = '--', linewidth = 3)

# plt.xlim([5,20])
# plt.ylim([-1,0])
plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')


# plt.plot(energy, np.log10(kerr.Q_1pP),linewidth=2, color = 'forestgreen')
# plt.plot(energy, np.log10(kerr_ch.Q_1pP),linewidth=2, color = 'red')

In [ ]:
## 
## Compare the polynomial to chebyshev fit 
##

energy3plot = np.arange(1, 10001, 1)


plt.plot((energy3plot), 
                   (pfit_Q3pE_8deg(np.log10(energy3plot))),
                   linewidth = 4, linestyle = '-', color = 'black'
                 )
plt.plot((energy3plot), 
                   (np.log10(np.exp(Cheb(energy3plot, cfit_Q3pE_8deg, len(cfit_Q3pE_8deg)-1, e1_E, e2_E)))),
                   linewidth = 4, linestyle = '--', color = 'tomato',
                 )
plt.plot(kerr.energy, np.log10(kerr.Q_3pE),linestyle = '-',linewidth=4, color = 'goldenrod')
plt.plot(iaea93_Q_3pE.energy, np.log10(iaea93_Q_3pE.Q_3pE), marker = 'd', markersize = 5, color = 'dodgerblue')


plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')
plt.ylim([-3, 3])

---
### <b style="color:blue"> Do the fits work? </b>

***Compare to other values cross sections***

In [ ]:
# energy = np.arange(1.5e1/1e3, 100, 1e-3)
energy = np.arange(1, 8001, 1)
cs = OZpy.CrossSections.CrossSec(energy)

f95 = cs.cs_fang95()
bw99 = cs.cs_bw99()
kerr = cs.cs_kerr_poly()
kerr_ch = cs.cs_kerr_cheb()

> - Q_3pP

In [ ]:
##
## Compare Fang et al 1995, Brosius & Woodgate 1999, and my fits
##
# plt.plot(energy, np.log10(f95.Q_3pP), color = 'dodgerblue')
# plt.plot(energy, np.log10(bw99.Q_3pP), color = 'tomato')
plt.plot(energy, np.log10(kerr.Q_3pP), color = 'black', linewidth = 3)
plt.plot(energy, np.log10(kerr.Q_3pP_alt), color = 'goldenrod', linestyle = '--', linewidth = 3)

plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')
# plt.ylim([-2, 1])

> - Q_3pE

In [ ]:
##
## Compare Fang et al 1995, Brosius & Woodgate 1999, and my fits
##
# plt.plot(energy, np.log10(f95.Q_3pE), color = 'dodgerblue')
# plt.plot(energy, np.log10(bw99.Q_3pE), color = 'tomato')
plt.plot(energy, np.log10(kerr.Q_3pE), color = 'black', linewidth = 3)
plt.plot(energy, np.log10(kerr_ch.Q_3pE), color = 'goldenrod', linestyle = '--', linewidth = 3)

plt.xlabel('Energy [keV]')
plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
plt.xscale('log')
# plt.ylim([-1, 1.5])

> - Q_3pH

In [ ]:
# ##
# ## Compare Fang et al 1995, Brosius & Woodgate 1999, and my fits
# ##
# plt.plot(energy, np.log10(f95.Q_13H), color = 'orchid')
# # plt.plot(energy, np.log10(bw99.Q_12E), color = 'tomato')
# plt.plot(energy, np.log10(kerr.Q_13H), color = 'black', linewidth = 3)
# plt.plot(energy, np.log10(kerr_ch.Q_13H), color = 'goldenrod', linestyle = '--', linewidth = 3)

# plt.plot(energy[1:], np.log10(kerr.Q_13H_3s[1:]), color = 'tomato', linewidth = 3)
# plt.plot(energy[1:], np.log10(kerr_ch.Q_13H_3s[1:]), color = 'pink', linestyle = '--', linewidth = 3)

# plt.plot(energy, np.log10(kerr.Q_13H_3p), color = 'limegreen', linewidth = 3)
# plt.plot(energy, np.log10(kerr_ch.Q_13H_3p), color = 'green', linestyle = '--', linewidth = 3)

# plt.plot(energy, np.log10(kerr.Q_13H_3d), color = 'skyblue', linewidth = 3)
# plt.plot(energy, np.log10(kerr_ch.Q_13H_3d), color = 'darkblue', linestyle = '--', linewidth = 3)



# plt.xlabel('Energy [keV]')
# plt.ylabel('log$_{10}$ Q [10$^{-17}$ cm$^{2}$]')
# plt.xscale('log')
# # plt.ylim([-1, 1.5])



In [ ]:
savepng = True
fname_out = 'MyFits_Q_3p_all'
dir1 = './'

logy = True
logx = True

csid1 = 'H(n=3)* + p = p* + p + e'
csid2 = 'H(n=3)* + H = p* + H + e'
csid3 = 'H(n=3)* + e = p* + e + e'


xrange = [1, 8000]
yrange = [1e-3, 5000]
cmaptemp = pal.cartocolors.sequential.Sunset_5.get_mpl_colormap()
colors = cmaptemp(np.linspace(0.1,1.0,5))


labels = ['Q_3pP (IEAE)', 'Q_3pH (poly)', 'Q_3pE (IEAE)', 'Q_3pP (cheb)', 'Q_3pH (cheb)', 'Q_3pE (cheb)']

if logx == True:
    xtitle = 'E [keV]'
elif logy == False:
    xtitle = 'E [keV]'
if logy == True:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
elif logy == False:
    ytitle = 'Q [10$^{-17}$ cm$^{2}$]'
    
xsize = 8
ysize = 5
fig = plt.figure(figsize=(xsize, ysize))

x1 = 0.075
y1 = 0.075
dx = 0.70
sx = 0.05
dy = 0.70
sy = 0.05

ax1 = fig.add_axes([x1, y1, dx, dy])

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=True,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True,
    labeltop = False,
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelright=False,
    labelleft = True
    ) 

ax1.spines["left"].set_position(("axes",-0.05))
ax1.spines["bottom"].set_position(("axes",-.075))
ax1.grid(True, axis='x', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)
ax1.grid(True, axis='y', which='both', linestyle = '-', linewidth = 0.25, color = 'grey', alpha = 0.25)

ax1.set_ylim(yrange[0],yrange[-1])
ax1.set_xlim(xrange[0],xrange[-1])
ax1.set_ylabel(ytitle)
ax1.set_xlabel(xtitle)


ax1.text(0.75, 0.95, csid1, color = 'black',transform=ax1.transAxes,fontsize = 16)   
ax1.text(0.75, 0.875, csid2, color = 'black',transform=ax1.transAxes,fontsize = 16)     
ax1.text(0.75, 0.80, csid3, color = 'black',transform=ax1.transAxes,fontsize = 16)     


# ax1.text(0.025, 0.80, 't = %0.2fs' %(timer[snap]), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.70, r'$\tau_{exp}$ = %0.2fs' %(exptime), color = 'black',transform=ax1.transAxes,fontsize = 16)     
# ax1.text(0.025, 0.60, '$\lambda_{0}$ = %0.2f$\mathrm{\AA}$'%(lambda_rest), color = 'black',transform=ax1.transAxes,fontsize = 16)     

if logx == True:
    ax1.set_xscale('log')
    ax1.xaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.xaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logx == False:
    ax1.set_xscale = 'linear'
if logy == True:
    ax1.set_yscale('log')
    ax1.yaxis.set_minor_locator(LogLocator(numticks=15,subs=np.arange(2,10))) #(2)
    for label in ax1.yaxis.get_ticklabels()[::5]:
        label.set_visible(False) #(3)
elif logy == False:
    ax1.set_yscale = 'linear'
        
line1, = ax1.plot(energy, (kerr.Q_3pP), color = 'skyblue',
                   linewidth = 3, linestyle = '-',
#                   zorder = 0, alpha = 0.35,
#                    marker = 'P', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[0]
                 )
# line2, = ax1.plot(energy, (kerr.Q_3pH), color = 'pink',
#                    linewidth = 3, linestyle = '-',
# #                   zorder = 50, alpha = 1,
# #                    marker = 'd', markeredgewidth = 1, 
# #                    markersize = 5,
#                    label = labels[1]
#                  )
line3, = ax1.plot(energy, (kerr.Q_3pE), color = 'black',
                   linewidth = 3, linestyle = '-',
#                   zorder = 0, alpha = 0.35,
#                    marker = 'X', markeredgewidth = 1, 
#                    markersize = 5,
                   label = labels[2]
                 )
line4, = ax1.plot(energy, (kerr.Q_3pP_alt), color = 'darkblue',
                   linewidth = 3, linestyle = '--',
#                   zorder = 0, alpha = 0.35,
#                    marker = 'D', markeredgewidth = 1, 
#                    markersize = 5,
                   label = labels[3]
                 )
# line5, = ax1.plot(energy, (kerr.Q_3pH), color = 'red',
#                    linewidth = 3, linestyle = '--',
# #                   zorder = 0, alpha = 0.35,
# #                    marker = 'o', markeredgewidth = 1, 
# #                    markersize = 8,
#                    label = labels[4]
#                  )
line6, = ax1.plot(energy, (kerr.Q_3pE_alt), color = 'lightgrey',
                   linewidth = 3, linestyle = '--',
#                   zorder = 0, alpha = 0.35,
#                    marker = 's', markeredgewidth = 1, 
#                    markersize = 8,
                   label = labels[5]
                 )




# leg1 = ax1.legend(handles=[line1,line2,line3,line4,line5,line6],loc=(1.025,-0.10), fontsize = 15)
leg1 = ax1.legend(handles=[line1,line3,line4,line6],loc=(1.025,-0.10), fontsize = 15)


# ax1.axvline(x=16, color = 'tomato', alpha = 0.5, linestyle = ':')

if logx == True:
    appendx = '_logx'
elif logx == False:
    appendx = '_linx'
if logy == True:
    appendy = '_logy'
elif logy == False:
    appendy = '_liny'
if savepng == True:
        plt.savefig(dir1+fname_out+appendx+appendy+'.png', format='png', bbox_inches = 'tight', dpi=300)

plt.show()